In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
import joblib
import numpy as np

In [7]:
f0 = open("1661-0.txt", "r")
f1 = open('pg5200.txt', 'r')
lines0 = [i for i in f0]
lines1 = [i for i in f1]
lines = lines0 + lines1

In [8]:
print("The First Line: ", lines[1])
print("The Last Line: ", lines[-1])

The First Line:  Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle

The Last Line:  subscribe to our email newsletter to hear about new eBooks.



In [19]:
words = ""

for i in lines:
  words = ' '.join(c for c in lines if not c.isdigit()) # converting all the lines in the text files to a corpus or text document
    
words = words.replace('\n', '').replace('\ufeff', '').replace("\\", '')
words[:300]

" Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle  This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or "

In [20]:
tokenizer = Tokenizer() #convert the words to a a sequences of integers
tokenizer.fit_on_texts([words]) #fit on our corpus
sequences = tokenizer.texts_to_sequences([words])[0]
len(sequences)

136561

In [14]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

9648


In [16]:
#we create training data based on each word in our array of sequences and the next word to follow
training_data = []
for i in range(1, len(sequences)):
    training_data.append(sequences[i-1:i+1])
print(training_data[:6])

[[101, 5442], [5442, 1], [1, 1215], [1215, 4], [4, 169], [169, 42]]


In [25]:
X = np.array([i[0] for i in training_data])
y = np.array([i[1] for i in training_data])
print(f"If X:{X[0]}, then y:{y[0]}")
y = keras.utils.to_categorical(y, num_classes=vocab_size)

If X:101, then y:5442


In [29]:
#Defining the model, a recurrent neural network
next_words_model = keras.models.Sequential([
                                            keras.layers.Embedding(vocab_size, 10, input_length=1),
                                            keras.layers.LSTM(1000, return_sequences=True),
                                            keras.layers.LSTM(1000),
                                            keras.layers.Dense(500, activation = 'relu'),
                                            keras.layers.Dense(vocab_size, activation = 'softmax')
])
next_words_model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

history = next_words_model.fit(X, y, epochs=50, batch_size=128, validation_split=0.05)

Epoch 1/50
325/325 [==============================] - 152s 461ms/step - loss: 6.7798 - accuracy: 0.0515 - val_loss: 6.8655 - val_accuracy: 0.0513
Epoch 2/50
325/325 [==============================] - 149s 458ms/step - loss: 6.4210 - accuracy: 0.0516 - val_loss: 6.8643 - val_accuracy: 0.0513
Epoch 3/50
325/325 [==============================] - 149s 458ms/step - loss: 6.2628 - accuracy: 0.0561 - val_loss: 6.4813 - val_accuracy: 0.0767
Epoch 4/50
325/325 [==============================] - 149s 458ms/step - loss: 5.9526 - accuracy: 0.0807 - val_loss: 6.1818 - val_accuracy: 0.0986
Epoch 5/50
325/325 [==============================] - 149s 459ms/step - loss: 5.6526 - accuracy: 0.1070 - val_loss: 5.9670 - val_accuracy: 0.1258
Epoch 6/50
325/325 [==============================] - 149s 458ms/step - loss: 5.4360 - accuracy: 0.1215 - val_loss: 5.8856 - val_accuracy: 0.1353
Epoch 7/50
325/325 [==============================] - 149s 458ms/step - loss: 5.2875 - accuracy: 0.1299 - val_loss: 5.8208 -

In [30]:
next_words_model.save('next_words_model.h5')
joblib.dump(tokenizer, 'text_tokenizer.joblib')

['text_tokenizer.joblib']

In [41]:
# Load the model and tokenizer

model = keras.models.load_model('next_words_model.h5')
tokenizer = joblib.load('text_tokenizer.joblib')

In [100]:
def retrain_model(text):
  retrain_seq = tokenizer.texts_to_sequences([text])[0]
  retrain_data = []
  for i in range(1, len(retrain_seq)):
    retrain_data.append(retrain_seq[i-1:i+1])
  X_train = np.array([v[0] for v in retrain_data])
  y_train = np.array([v[1] for v in retrain_data])
  y_train = keras.utils.to_categorical(y_train, num_classes=vocab_size)
  model.fit(X_train, y_train, epochs=10)

In [116]:
def predict_next_word(text):
  try:
    text = text.split(" ")
    text = text[-1]
  
  except:
    print('Invalid Entry')

  sequence = tokenizer.texts_to_sequences([text])[0]
  sequence = np.array(sequence)

  preds = model.predict(sequence)
  pred_words = tokenizer.sequences_to_texts([sorted(np.argsort(preds)[0][-3:])]) #getting the words with the highest probabilities

  pred_words = pred_words[0].split(' ')
  print("The most probable word is: {} \n Other likely words: {}, {}".format(pred_words[0], pred_words[1], pred_words[2]))

In [ ]:
retrain_model('My hand hurts like hell')

In [118]:
predict_next_word('hand')

The most probable word is: is 
 Other likely words: lucy, hurts
